# HCPCS-4: HCPCS first char is valid

Description: check if the first char of all HCPCS codes are valid. The valid first char of HCPCS code could be: 
('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'G', 'H', 'J', 'K', 'L',
               'M', 'P', 'Q', 'R', 'S', 'T', 'V').

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [5]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out all failed HCPCS codes.


def hcpcs_4(db_name, table_name, col_name):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        print ('Test file: {}.{}'.format(db_name, table_name))
        print ('\n')

        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE   {col1} NOT LIKE '0%' 
                AND {col1} NOT LIKE '1%' 
                AND {col1} NOT LIKE '2%' 
                AND {col1} NOT LIKE '3%' 
                AND {col1} NOT LIKE '4%' 
                AND {col1} NOT LIKE '5%' 
                AND {col1} NOT LIKE '6%' 
                AND {col1} NOT LIKE '7%' 
                AND {col1} NOT LIKE '8%' 
                AND {col1} NOT LIKE '9%' 
                AND {col1} NOT LIKE 'A%' 
                AND {col1} NOT LIKE 'B%' 
                AND {col1} NOT LIKE 'C%' 
                AND {col1} NOT LIKE 'D%' 
                AND {col1} NOT LIKE 'E%' 
                AND {col1} NOT LIKE 'G%' 
                AND {col1} NOT LIKE 'H%' 
                AND {col1} NOT LIKE 'J%' 
                AND {col1} NOT LIKE 'K%' 
                AND {col1} NOT LIKE 'L%' 
                AND {col1} NOT LIKE 'M%' 
                AND {col1} NOT LIKE 'P%' 
                AND {col1} NOT LIKE 'Q%' 
                AND {col1} NOT LIKE 'R%' 
                AND {col1} NOT LIKE 'S%' 
                AND {col1} NOT LIKE 'T%' 
                AND {col1} NOT LIKE 'V%'
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))
        
        if not len(rows):
            print ("Test result: PASS")
        else:
            print ("Test result: FAIL" + '\n')
            print ("The following {} fail the test:".format(col_name) + '\n')
            
            # MySQL query to find all bad HCPCS codes
            query = ('''
                SELECT {col1}
                FROM {db}.{t1}
                WHERE   {col1} NOT LIKE '0%' 
                    AND {col1} NOT LIKE '1%' 
                    AND {col1} NOT LIKE '2%' 
                    AND {col1} NOT LIKE '3%' 
                    AND {col1} NOT LIKE '4%' 
                    AND {col1} NOT LIKE '5%' 
                    AND {col1} NOT LIKE '6%' 
                    AND {col1} NOT LIKE '7%' 
                    AND {col1} NOT LIKE '8%' 
                    AND {col1} NOT LIKE '9%' 
                    AND {col1} NOT LIKE 'A%' 
                    AND {col1} NOT LIKE 'B%' 
                    AND {col1} NOT LIKE 'C%' 
                    AND {col1} NOT LIKE 'D%' 
                    AND {col1} NOT LIKE 'E%' 
                    AND {col1} NOT LIKE 'G%' 
                    AND {col1} NOT LIKE 'H%' 
                    AND {col1} NOT LIKE 'J%' 
                    AND {col1} NOT LIKE 'K%' 
                    AND {col1} NOT LIKE 'L%' 
                    AND {col1} NOT LIKE 'M%' 
                    AND {col1} NOT LIKE 'P%' 
                    AND {col1} NOT LIKE 'Q%' 
                    AND {col1} NOT LIKE 'R%' 
                    AND {col1} NOT LIKE 'S%' 
                    AND {col1} NOT LIKE 'T%' 
                    AND {col1} NOT LIKE 'V%';
            '''.format(db = db_name, t1 = table_name, col1 = col_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [6]:
hcpcs_4('_amy', 'test_data_good','hcpcs_cd')

Test file: _amy.test_data_good


Test result: PASS


In [7]:
hcpcs_4('_amy', 'test_data_bad1','hcpcs_cd')

Test file: _amy.test_data_bad1


Test result: FAIL

The following hcpcs_cd fail the test:

F0, 

, 

, 

, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

, 

, 

, 

, 

F0, 

, 

, 

, 

F0, 

F0, 

, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

, 

F0, 

F0, 

, 

, 

, 

F0, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

, 

F0, 

F0, 

, 

F0, 

, 

F0, 

, 

F0, 

F0, 

, 

, 

, 

F0, 

F0, 

, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

F0, 

, 

, 

, 

, 

F0, 

, 

F0, 

F0, 

F0, 

F0, 

F0, 

, 

F0, 

, 

F0, 

F0, 

F0, 

, 

, 

F0, 

F0, 

F0, 

F0, 

F0, 

, 

, 

, 

F0, 

, 

, 

F0, 

, 

F0, 

F0, 

F0, 

